<a href="https://colab.research.google.com/github/shlok-py/ML_DL_programs/blob/main/3DConvolutionforVideoClassificationusingC3Dnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install remotezip tqdm opencv-python einops
!pip install -U tensorflow keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers

In [ ]:
import requests
import os

def download_file(url, save_path):

    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

def download_ucf_dataset():

    base_url = "https://storage.googleapis.com/thumos14_files/UCF101_videos.zip"

    save_dir = "./"
    save_path = "./UCF101_videos.zip"

    tqdm.tqdm(download_file(base_url, save_path))

if __name__ == "__main__":
    download_ucf_dataset()


0it [00:00, ?it/s]


In [ ]:
!unzip UCF101_videos.zip

Streaming output truncated to the last 5000 lines.
  inflating: UCF101/v_JavelinThrow_g17_c02.avi  
  inflating: UCF101/v_JavelinThrow_g17_c03.avi  
  inflating: UCF101/v_JavelinThrow_g17_c04.avi  
  inflating: UCF101/v_JavelinThrow_g17_c05.avi  
  inflating: UCF101/v_JavelinThrow_g18_c01.avi  
  inflating: UCF101/v_JavelinThrow_g18_c02.avi  
  inflating: UCF101/v_JavelinThrow_g18_c03.avi  
  inflating: UCF101/v_JavelinThrow_g18_c04.avi  
  inflating: UCF101/v_JavelinThrow_g19_c01.avi  
  inflating: UCF101/v_JavelinThrow_g19_c02.avi  
  inflating: UCF101/v_JavelinThrow_g19_c03.avi  
  inflating: UCF101/v_JavelinThrow_g19_c04.avi  
  inflating: UCF101/v_JavelinThrow_g20_c01.avi  
  inflating: UCF101/v_JavelinThrow_g20_c02.avi  
  inflating: UCF101/v_JavelinThrow_g20_c03.avi  
  inflating: UCF101/v_JavelinThrow_g20_c04.avi  
  inflating: UCF101/v_JavelinThrow_g21_c01.avi  
  inflating: UCF101/v_JavelinThrow_g21_c02.avi  
  inflating: UCF101/v_JavelinThrow_g21_c03.avi  
  inflating: UCF10

In [ ]:
dataset_dir = "./UCF101"
len(os.listdir(dataset_dir))

In [ ]:
import re
import shutil
def make_classes(dir):
    for files in os.listdir(dir):
            pattern = r'_(.*?)_'  # Regex pattern to match text between underscores
            class_name = re.findall(pattern, files)[0]
            os.makedirs(os.path.join(dir, class_name), exist_ok = True)
            shutil.move(os.path.join(dir,files), os.path.join(dir, class_name, files))
            print(f"{files} moves from {os.path.join(dir,files)} to {os.path.join(dir, class_name, files)}\n")
make_classes('./UCF101')

import random

def reduce_classes(dir, num_classes_to_keep=10):
    # List all class directories
    class_dirs = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]

    # Check if we have enough classes
    if len(class_dirs) < num_classes_to_keep:
        raise ValueError(f"Not enough classes to reduce. {len(class_dirs)} classes.")

    # Randomly select classes to keep
    classes_to_keep = random.sample(class_dirs, num_classes_to_keep)

    # Remove the other class directories
    for class_dir in class_dirs:
        if class_dir not in classes_to_keep:
            shutil.rmtree(os.path.join(dir, class_dir))
            print(f"Removed class directory: {class_dir}")

    print(f"Kept the following {num_classes_to_keep} classes: {', '.join(classes_to_keep)}")

# Usage
reduce_classes('./UCF101', num_classes_to_keep=10)

import os
import random
import shutil

def split_dataset(dir, train_ratio=0.8):
    # List all class directories
    class_dirs = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]

    for class_dir in class_dirs:
        video_files = [f for f in os.listdir(os.path.join(dir, class_dir)) if os.path.isfile(os.path.join(dir, class_dir, f))]

        random.shuffle(video_files)

        num_files = len(video_files)
        split_index = int(train_ratio * num_files)

        train_files = video_files[:split_index]
        val_files = video_files[split_index:]

        train_dir = os.path.join(dir, class_dir, 'train')
        val_dir = os.path.join(dir, class_dir, 'val')
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)

        # Move files to train directory
        for file in train_files:
            shutil.move(os.path.join(dir, class_dir, file), os.path.join(train_dir, file))

        # Move files to val directory
        for file in val_files:
            shutil.move(os.path.join(dir, class_dir, file), os.path.join(val_dir, file))

        print(f"Split {class_dir} into train and val sets with {train_ratio*100}% for train.")


split_dataset('./UCF101')


In [ ]:
dict((name, idx) for idx, name in enumerate(os.listdir(dataset_dir)))

{'Lunges': 0,
 'TennisSwing': 1,
 'WritingOnBoard': 2,
 'CuttingInKitchen': 3,
 'BlowDryHair': 4,
 'SumoWrestling': 5,
 'Biking': 6,
 'TrampolineJumping': 7,
 'BlowingCandles': 8,
 'JugglingBalls': 9}

In [ ]:
# import random
# class FrameGenerator:
#     def __init__(self, path, width, height, depth, dataset = dataset_dir, color = True, skip = True, training = True):
#         self.path = pathlib.Path(path)
#         self.width = width
#         self.height = height
#         self.depth = depth
#         self.color = color
#         self.skip = skip
#         self.dataset = dataset_dir
#         self.training = training
#         # self.nframe  = nframe
#         self.class_names = os.listdir(dataset)
#         self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))


#     def video3d_frames(self, filepath):
#         cap = cv2.VideoCapture(filepath)
#         nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)

#         if self.skip:
#             frames = [x * nframe / self.depth for x in range(self.depth)]
#         else:
#             frames = [x for x in range(self.depth)]

#         framearray = []
#         for i in range(self.depth):
#             cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
#             ret, frame = cap.read()
#             frame = cv2.resize(frame, (self.height, self.width), interpolation = cv2.INTER_CUBIC)
#             if self.color:
#                 framearray.append(frame)
#             else:
#                 framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

#         cap.release()

#         X = np.array(framearray)    #vid3d.video3d(video_dir, color=color, skip=skip)
#         print("CV2 video shape:",X.shape)

#         if self.color:
#             out = np.array(X).transpose((1, 2, 0, 3))
#             print("Final video shape:",out.shape)
#             return out

#         else:
#             return np.array(X).transpose((1, 2, 0, 0))

#     def files_and_class (self):
#         video_paths = list(self.path.glob('*/*.avi'))
#         classes = [p.parent.name for p in video_paths]
#         return video_paths, classes

#     def __call__(self):
#         video_paths, classes = self.files_and_class()

#         pairs = list(zip(video_paths, classes))

#         if self.training:
#             random.shuffle(pairs)

#         for path, name in pairs:
#             video_frames = self.video3d_frames(str(path))
#             label = self.class_ids_for_name[name] # Encode labels
#             yield video_frames, label
import numpy as np
import random
from pathlib import Path
from tensorflow.keras.utils import Sequence, to_categorical
import cv2

# import numpy as np
# import random
# from pathlib import Path
# import cv2

class FrameGenerator(Sequence):
    def __init__(self, path, width, height, depth, batch_size=4, color=False, skip=True, training=True):
        self.path = Path(path)
        self.width = width
        self.height = height
        self.depth = depth
        self.color = color
        self.skip = skip
        self.batch_size = batch_size
        self.training = training
        self.classes = sorted(os.listdir(self.path))
        self.class_ids_for_name = {name: idx for idx, name in enumerate(self.classes)}
        self.num_classes = len(self.classes)
        self.file_paths = self._load_file_paths()
        self.on_epoch_end()

    def _load_file_paths(self):
        file_paths = {}
        for class_name in self.classes:
            if self.training:
                file_dir = self.path / class_name / 'train'
            else:
                file_dir = self.path / class_name / 'val'

            file_paths[class_name] = list(file_dir.glob('*.avi'))
        return file_paths

    def __len__(self):
        total_files = sum(len(files) for files in self.file_paths.values())
        return total_files // self.batch_size

    def __getitem__(self, idx):
        batch_X, batch_y = [], []
        all_files = [(class_name, file) for class_name, files in self.file_paths.items() for file in files]
        batch_files = all_files[idx * self.batch_size:(idx + 1) * self.batch_size]

        for class_name, file in batch_files:
            video_frames = self.video3d_frames(file)
            label = self.class_ids_for_name[class_name]
            batch_X.append(video_frames)
            batch_y.append(label)

        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        batch_y = to_categorical(batch_y, num_classes=self.num_classes)

        return batch_X, batch_y

    def video3d_frames(self, filepath):
        cap = cv2.VideoCapture(str(filepath))
        nframe = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if self.skip:
            frames = [int(x * nframe / self.depth) for x in range(self.depth)]
        else:
            frames = list(range(self.depth))

        framearray = []
        for i in frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_CUBIC)
            if self.color:
                framearray.append(frame)
            else:
                framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()

        X = np.array(framearray)
        if self.color:
            return X.transpose((1, 2, 0, 3))  # (H, W, D, C)
        else:
            return X.transpose((1, 2, 0))  # (H, W, D)

    def on_epoch_end(self):
        if self.training:
            for class_name in self.classes:
                files = list(self.path.joinpath(class_name, 'train').glob('*.avi'))
                random.shuffle(files)
                self.file_paths[class_name] = files
        else:
            for class_name in self.classes:
                files = list(self.path.joinpath(class_name, 'val').glob('*.avi'))
                self.file_paths[class_name] = files

# Usage example
frame_gen = FrameGenerator(path='./UCF101', width=128, height=128, depth=20, batch_size=4, color=True, training=True)
val = FrameGenerator(path='./UCF101', width=128, height=128, depth=20, batch_size=4, color=True, training=False)


In [ ]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, ZeroPadding3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

def C3D(weights='msd', input_tensor=None, include_top=True):
    if weights not in {'msd', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `msd` '
                         '(pre-training on Million Song Dataset).')

    input_shape = (128, 128, 20, 3)

    if input_tensor is None:
        melgram_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            melgram_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            melgram_input = input_tensor

    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same', name='conv1', strides=(1, 1, 1))(melgram_input)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool1')(x)

    x = Conv3D(128, (3, 3, 3), activation='relu', padding='same', name='conv2', strides=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool2')(x)

    x = Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3a', strides=(1, 1, 1))(x)
    x = Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3b', strides=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool3')(x)

    x = Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4a', strides=(1, 1, 1))(x)
    x = Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4b', strides=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool4')(x)

    x = Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5a', strides=(1, 1, 1))(x)
    x = Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5b', strides=(1, 1, 1))(x)
    x = ZeroPadding3D(padding=(0, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool5')(x)

    x = Flatten()(x)

    if include_top:
        x = Dense(4096, activation='relu', name='fc6')(x)
        x = Dropout(0.5)(x)
        x = Dense(4096, activation='relu', name='fc7')(x)
        x = Dropout(0.5)(x)
        x = Dense(10, activation='softmax', name='fc8')(x)

    model = Model(inputs=melgram_input, outputs=x, name='C3D')

    return model

# Compile the model
model = C3D()
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "C3D"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 20, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv3D)                       │ (None, 128, 128, 20, 64)    │           5,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling3D)                 │ (None, 128, 64, 10, 64)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv3D)                       │ (None, 128, 64, 10, 128)    │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling3D)                 │ (None, 128, 32, 5, 128)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3a (Conv3D)                      │ (None, 128, 32, 5, 256)     │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3b (Conv3D)                      │ (None, 128, 32, 5, 256)     │       1,769,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool3 (MaxPooling3D)                 │ (None, 64, 16, 2, 256)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv4a (Conv3D)                      │ (None, 64, 16, 2, 512)      │       3,539,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv4b (Conv3D)                      │ (None, 64, 16, 2, 512)      │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool4 (MaxPooling3D)                 │ (None, 64, 8, 1, 512)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv5a (Conv3D)                      │ (None, 64, 8, 1, 512)       │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv5b (Conv3D)                      │ (None, 64, 8, 1, 512)       │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ zero_padding3d (ZeroPadding3D)       │ (None, 64, 10, 3, 512)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool5 (MaxPooling3D)                 │ (None, 32, 5, 1, 512)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 81920)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc6 (Dense)                          │ (None, 4096)                │     335,548,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc7 (Dense)                          │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc8 (Dense)                          │ (None, 10)                  │          40,970 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 380,026,634 (1.42 GB)

 Trainable params: 380,026,634 (1.42 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
import gc

# Clear session and run garbage collection
tf.keras.backend.clear_session()
gc.collect()

0

In [ ]:
history = model.fit(frame_gen,
                    validation_data=val,
                    epochs=20,
                    verbose=1)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


255/255 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - accuracy: 0.1104 - loss: 2.9165 - val_accuracy: 0.1346 - val_loss: 2.2920
Epoch 2/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 402s 1s/step - accuracy: 0.1225 - loss: 2.3001 - val_accuracy: 0.2154 - val_loss: 2.1858
Epoch 3/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step - accuracy: 0.2082 - loss: 2.2411 - val_accuracy: 0.2115 - val_loss: 2.1106
Epoch 4/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 388s 2s/step - accuracy: 0.2252 - loss: 2.1833 - val_accuracy: 0.2462 - val_loss: 2.1179
Epoch 5/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 366s 1s/step - accuracy: 0.2526 - loss: 2.1017 - val_accuracy: 0.2385 - val_loss: 2.0812
Epoch 6/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.2309 - loss: 2.1246 - val_accuracy: 0.2192 - val_loss: 2.1183
Epoch 7/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 387s 1s/step - accuracy: 0.2410 - loss: 2.0409 - val_accuracy: 0.3000 - val_loss: 1.7598
Epoch 8/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 363s 1s/step - accuracy: 0.3729 - loss: 1.7992 - val_accuracy: 0.373

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()